# Generation-based MRC 문제를 풀어보기

In [1]:
# Requirements
# !pip install datasets==1.4.1
# !pip install transformers==4.4.1
!pip install sentencepiece==0.1.95
!pip install nltk

distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 데이터 및 평가 지표 불러오기

In [3]:
from datasets import load_dataset

datasets = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [4]:
from datasets import load_metric

metric = load_metric('squad')

## Pre-trained 모델 및 토크나이저 불러오기

In [5]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

In [6]:
model_name = "google/mt5-small"

In [7]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config,
    cache_dir=None,
)

## 설정하기

In [8]:
max_source_length = 1024
max_target_length = 128
padding = False
preprocessing_num_workers=12
num_beams = 2
max_train_samples = 16
max_val_samples = 16
batch_size = 4
num_train_epochs = 2

## 전처리하기

In [9]:
def preprocess_function(examples):
    inputs = [f'question: {q}  context: {c} </s>' for q, c in zip(examples['question'], examples['context'])]
    targets = [f'{a["text"][0]} </s>' for a in examples['answers']]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["example_id"] = []
    for i in range(len(model_inputs["labels"])):
        model_inputs["example_id"].append(examples["id"][i])
    return model_inputs

In [10]:
column_names = datasets['train'].column_names

In [11]:
# TODO: Train mt5 on whole KorQuAD datset

train_dataset = datasets["train"]
# train_dataset = train_dataset.select(range(max_train_samples))
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

In [12]:
# TODO: Train mt5 on whole KorQuAD datseteval_dataset = datasets["validation"]

eval_dataset = datasets["validation"]
# eval_dataset = eval_dataset.select(range(max_val_samples))
eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )


## Fine-tuning하기

In [13]:
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

In [14]:
label_pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id,
            pad_to_multiple_of=None,
        )

In [15]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # decoded_labels is for rouge metric, not used for f1/em metric
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    formatted_predictions = [{"id": ex['id'], "prediction_text": decoded_preds[i]} for i, ex in enumerate(datasets["validation"].select(range(max_val_samples)))]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"].select(range(max_val_samples))]

    result = metric.compute(predictions=formatted_predictions, references=references)
    return result

In [16]:
args = Seq2SeqTrainingArguments(
    output_dir='/mnt/miyoung/outputs', 
    do_train=True, 
    do_eval=True, 
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_train_epochs,
    save_strategy='epoch'
)

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [18]:
train_result = trainer.train(resume_from_checkpoint=None)

Step,Training Loss
500,16.332000
1000,1.957700
1500,1.146900
2000,0.843600
2500,0.661200
3000,0.557900
3500,0.499900
4000,0.448200
4500,0.409600
5000,0.381100


In [19]:
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     '/mnt/miyoung/outputs/checkpoint-30204',
#     config=config,
#     cache_dir=None,
# )

## 평가하기

In [20]:
metrics = trainer.evaluate(
    max_length=max_target_length, num_beams=num_beams, metric_key_prefix="eval"
)

In [21]:
metrics

{'eval_loss': 0.18836341798305511,
 'eval_exact_match': 43.75,
 'eval_f1': 68.69047619047619,
 'eval_runtime': 311.6601,
 'eval_samples_per_second': 18.527,
 'epoch': 2.0,
 'eval_mem_cpu_alloc_delta': -25571328,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 79290368,
 'eval_mem_gpu_peaked_delta': 1826576896}

In [26]:
document = "미국의 대통령은?"
input_ids = tokenizer(document, return_tensors='pt').input_ids
outputs = model.generate(input_ids.to('cuda'))

In [27]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

미국의 대통령 
